In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tqdm
import datetime
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

%load_ext tensorboard

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
class_indices = ['Colorless', 'Darkness', 'Fighting', 'Fire', 'Grass', 'Lightning', 'Metal', 'Psychic', 'Water']

In [ ]:
xception_model = tf.keras.applications.Xception(input_shape=(300,300,3), classes=9, weights=None)
xception_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
xception_model.load_weights('./pesos_boms/Xception_2020-08-04-nonbalanced-dataset/epoch-38-val_loss-1.71.tf')

In [ ]:
inception_model = tf.keras.applications.InceptionResNetV2(input_shape=(300,300,3), classes=9, weights=None)
inception_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
inception_model.load_weights('./pesos_boms/Inception-nonbalanced-dataset/epoch-24-val_loss-1.76.tf')

In [ ]:
test_images = []
test_labels = []

base_dir = './dataset/test_images'
for label in os.listdir(base_dir):
    label_dir = os.path.join(base_dir, label)
    for img in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img)
        
        img = np.array(cv2.imread(img_path, cv2.IMREAD_COLOR))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img/256.0
        
        test_images.append(img)
        test_labels.append(int(label))

test_images = np.array(test_images)

In [ ]:
xception_guess = []
inception_guess = []
combined_guess = []

xception_res = xception_model.predict(test_images)
inception_res = inception_model.predict(test_images)

for i in range(len(test_images)):
    xception_guess.append((np.argmax(xception_res[i]), np.max(xception_res[i])))
    inception_guess.append((np.argmax(inception_res[i]), np.max(inception_res[i])))
    
    if 1.1*xception_guess[i][1] > inception_guess[i][1]:
        combined_guess.append(xception_guess[i])
    else:
        combined_guess.append(inception_guess[i])

In [ ]:
xception_right = 0
inception_right = 0
combined_right = 0

for i in range(len(test_images)):
    if xception_guess[i][0] == test_labels[i]:
        xception_right += 1
    if inception_guess[i][0] == test_labels[i]:
        inception_right += 1
    if combined_guess[i][0] == test_labels[i]:
        combined_right += 1

print("Xception: ", xception_right/len(test_images))
print("Inception: ", inception_right/len(test_images))
print("Combined: ", combined_right/len(test_images))